In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.style as style
import os
from collections import deque

In [6]:
lightunit = "ppfd"
lightname = 'Light [{}]'.format(lightunit)
timename = "Time [hr]"
doublename = "Doubling Time [hr]"
maxname = "Max Growth Rate [doublings/hr]"
odtype = "680" # which OD to use
odname = "OD("+odtype+"nm)"
volume = 25 #vial culture volume

exptname = 'test_expt'
save_folder = exptname+'_analysis'

In [34]:
path = os.getcwd()
save_path = os.path.join(path,save_folder)
vial_nums = [3] # which vials to analyze
strain1 = 'PCC_11801'
strain2 = ''
vial_to_strain = {
           0: 0,
           1: 0,
           2: 0,
           3: 0,
           4: strain1,
           5: 0,
           6: 0,
           7: 0,
           8: 0,
           9: 0,
           10: 0,
           11: 0,
           12: 0,
           13: 0,
           14: 0,
           15: 0
          }

In [35]:
for x in vial_to_strain.keys():
    if vial_to_strain[x] == 0:
        continue
        
    od_path = "%s/%s/OD/vial%d_OD.txt" % (path, exptname, x)
    od_data = np.genfromtxt(od_path, delimiter=',')
    odx = od_data[:, 0]
    ody = od_data[:, 1]
    od_last = np.median(ody[-7:])
    
    
    light_path = "%s/%s/OD/vial%d_OD_light.txt" % (path, exptname, x)
    light_data = np.genfromtxt(light_path, delimiter=',')
    lightx = light_data[:, 0]
    lighty = light_data[:, 1]
    light_last = np.median(lighty[-7:])
    
    bubble_path = "%s/%s/OD/vial%d_OD_bubble.txt" % (path, exptname, x)
    light_data = np.genfromtxt(od_path, delimiter=',')
    lightx = od_data[:, 0]
    lighty = od_data[:, 1]
    light_last = np.median(ody[-7:])
    
    
    title = 'Smoothed Average OD'
    plt.figure(figsize=(8,5), dpi=150)
    palette = sns.color_palette("rainbow_r", lightcons)
    g = sns.lineplot(x=hourname, y=odname,
                 hue=lightname,
                 palette=palette,data=smooth).set_title(title)

[[            nan             nan]
 [ 3.40000000e-03  0.00000000e+00]
 [ 9.00000000e-03  1.61541800e-05]
 [ 1.46000000e-02  1.93852844e-05]
 [ 2.02000000e-02  9.69223967e-06]
 [ 2.57000000e-02  6.46140367e-06]
 [ 3.12000000e-02  1.93852844e-05]
 [ 3.68000000e-02 -1.29217341e-05]
 [ 4.23000000e-02  1.61541800e-05]
 [ 4.79000000e-02  1.93852844e-05]
 [ 5.35000000e-02  3.23105966e-05]
 [ 5.90000000e-02  2.58477616e-05]
 [ 6.46000000e-02  6.46140367e-06]
 [ 6.99000000e-02  9.69223967e-06]
 [ 7.58000000e-02  1.61541800e-05]
 [ 8.12000000e-02  1.29231651e-05]
 [ 8.67000000e-02  3.87737896e-05]
 [ 9.22000000e-02  0.00000000e+00]
 [ 9.77000000e-02  1.93852844e-05]
 [ 1.03500000e-01  1.93852844e-05]
 [ 1.09000000e-01  0.00000000e+00]
 [ 1.14600000e-01  1.61541800e-05]
 [ 1.20100000e-01  9.69223967e-06]
 [ 1.25700000e-01  2.26164782e-05]
 [ 1.31200000e-01  1.93852844e-05]
 [ 1.36600000e-01  2.58477616e-05]
 [ 1.42200000e-01  1.61541800e-05]
 [ 1.58900000e-01  1.93852844e-05]
 [ 1.81300000e-01  1

In [44]:
od_coefficients=[23349.078211301374, 100292385.68670042, -1.5563049794724384, -2.9791575207903462]
x=21611.0

np.real(od_coefficients[2] -
                            ((np.log10((od_coefficients[1] - od_coefficients[0]) /
                                        (float(x) - od_coefficients[0])-1)) /
                                        od_coefficients[3]))
#                     print("OD TRANSFORM:",x,od_coefficients,od_data[x])

<ipython-input-44-925d2b6931f4>:5: RuntimeWarning: invalid value encountered in log10
  ((np.log10((od_coefficients[1] - od_coefficients[0]) /


nan

In [45]:
#         100,292,385.           23,349.0             21,611      23,349
np.log10((od_coefficients[1] - od_coefficients[0]) / (float(x) - od_coefficients[0])-1)

<ipython-input-45-c657e868612f>:1: RuntimeWarning: invalid value encountered in log10
  np.log10((od_coefficients[1] - od_coefficients[0]) / (float(x) - od_coefficients[0])-1)


nan

In [51]:
a = od_coefficients[1] - od_coefficients[0]
a

100269036.60848911

In [52]:
b = float(x) - od_coefficients[0]
b

-1738.0782113013738

In [53]:
a/b

-57689.599902074246